In [23]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions
using MacroTools

In [24]:
@model function smoothing(n, k, x0, P)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)

    x_prev = x_prior
    
    sync = PostponeScheduler()

    for i in 1:n
        x[i] ~ (x_prev + 1.0) where { portal = rem(i, k) === 0 ? ScheduleOnPortal(sync) : EmptyPortal() }
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return sync, x, y
end

smoothing (generic function with 1 method)

In [33]:
P = 1.0

n = 100_000
k = 700
data = collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [34]:
function inference(; data, k, x0, P)
    n = length(data)
    
    _, (sync, x, y) = smoothing(n, k, x0, P);

    buffer    = Vector{Marginal}(undef, n)
    marginals = collectLatest(getmarginals(x))
    
    subscription = subscribe!(marginals, (ms) -> copyto!(buffer, ms))
    
    wait(sync)
    
    update!(y, data)
    
    wait(sync)
    
    unsubscribe!(subscription)
    
    return buffer
end

inference (generic function with 1 method)

In [35]:
@time res = inference(
    data = data,
    k = k,
    x0 = NormalMeanVariance(0.0, 10000.0),
    P = P
)

 12.884358 seconds (70.86 M allocations: 4.305 GiB, 53.69% gc time)


100000-element Array{Marginal,1}:
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=1.0000472539418537, v=9.999999990000086e-6))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=2.0000472539418537, v=9.999999990000086e-6))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=3.0000472539418537, v=9.999999990000084e-6))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=4.000047253941854, v=9.999999990000084e-6))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=5.0000472539418555, v=9.999999990000084e-6))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=6.000047253941855, v=9.999999990000086e-6))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=7.0000472539418555, v=9.999999990000084e-6))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=8.000047253941856, v=9.999999990000084e-6))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance